### Description

a simple jupyter notebook that helps Splunk Threat Research Team see the data connections, commonalities and differences of security content stories or threat families.This helps us to look for visualizing the common TTP’s and tooling used by adversaries or threat actors based on the Security content analytics we developed in each analytic story.


In [1]:
__author__ = "Teoderick Contreras STRT"
import os
import argparse
from pathlib import Path
import json
import csv
import yaml
from collections import defaultdict
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [2]:
SECURITY_CONTENT_PATH = "~/security_content/detections/"
sec_content_name_and_story = defaultdict(list)
mitre_attack_id_info_dict = defaultdict(list)
mitre_attack_tid_dict = defaultdict(list)
mitre_attack_fn_dict = defaultdict(list)
mitre_attack_story_dict = defaultdict(list)
mitre_attack_tid_name_dict = defaultdict(list)
home_path = Path.home()
story_detection_list = []

In [3]:
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

def expand_path(file_path):
    if "~" in file_path:
        return str(file_path).replace("~", str(home_path))
    else:
        return file_path

def generate_name_and_story_dict(sec_content_path, story):
    
    for roots, dirs, files in os.walk(sec_content_path):
        temp_file_name = ""
        #print("[+] enumerating yml files in: {0}".format(roots))
        detection_base_dir_name = os.path.basename(roots)
        
        for f in files:
            temp_file_name = os.path.join(detection_base_dir_name, f)
            file_path = os.path.join(roots, f)
            if file_path.endswith(".yml"):
                parse_yml_file(file_path, temp_file_name, story)
    print("[+] total detection found: {0}".format(len(story_detection_list)))
    return
    
def parse_yml_file(file_path, base_file_name, story):
    file_name = os.path.basename(base_file_name)
    mitre_tid = ""
    tid_json_obj = load_attack_id_json()
    with open(file_path, "r") as f:
        yml_buff = yaml.safe_load(f)
    
    for s in story:
        s = s.strip()
        if s in yml_buff['tags']['analytic_story']:

            sec_content_name_and_story[s].append(file_name)
            print("       [{0}] : {1}".format(s, base_file_name))
            story_detection_list.append(base_file_name)
            #f_name = "-".join(yml_buff['tags']["mitre_attack_id"]) + ":" + file_name 
            f_name = file_name


            mitre_attack_id_info_dict[f_name].append(s)
            for m in yml_buff['tags']["mitre_attack_id"]:

                mitre_attack_tid_dict[m].append(s)
                mitre_attack_tid_dict[m] = sorted(list(set(mitre_attack_tid_dict[m])))
                
                mitre_attack_fn_dict[m].append(s + ":" +file_name)
                mitre_attack_fn_dict[m] = sorted(list(set(mitre_attack_fn_dict[m])))
                

                mitre_attack_story_dict[s].append(m)
                mitre_attack_story_dict[s] = sorted(list(set(mitre_attack_story_dict[s])))
                
                new_key = m + "-" + tid_json_obj[m].replace(" ", "_")
                mitre_attack_tid_name_dict[new_key].append(s)
                mitre_attack_tid_name_dict[new_key] = sorted(list(set(mitre_attack_tid_name_dict[new_key])))
            #logger.info(self.mitre_attack_id_info_dict[file_name])
            #for mitre_tid in yml_buff['tags']['mitre_attack_id']:
            #    self.mitre_attack_tid_dict[mitre_tid].append(s)
        
    return

def load_attack_id_json():
    template_path = os.path.join(os.getcwd(), "templates")
    tid_json_path = os.path.join(template_path, "tid.json")
    with open(tid_json_path, "r") as f:
        tid_json_obj = json.load(f)
    return tid_json_obj

def process_sec_content(story_list):
        debug = 0
        print(" [+] story coverage needed: {0}".format(story_list))
        normalized_security_content_path = expand_path(SECURITY_CONTENT_PATH)
        generate_name_and_story_dict(normalized_security_content_path, story_list)
        if debug == 1:
            print("\n-----------------------------\n[-] detection name vs analytic story - mitre_attack_id_info_dict \n-----------------------------\n")
            for k, v in mitre_attack_id_info_dict.items():
                print("[{0}] detection name: {1} related story: {2}".format(len(v), k, v))    
            print("\n-----------------------------\n[-] mitre attack id vs analytic story - mitre_attack_tid_dict \n-----------------------------\n")
            for k, v in mitre_attack_tid_dict.items():
                print("[{0}] {1} {2}".format(len(list(set(v))), k, list(set(v))))
            print("\n-----------------------------\n[-] mitre attack id vs story:detectioname - mitre_attack_fn_dict \n-----------------------------\n")
            for k, v in mitre_attack_fn_dict.items():
                print("[{0}] {1} {2}".format(len(list(set(v))), k, list(set(v)))) 
            print("\n-----------------------------\n[-] story vs id - mitre_attack_story_dict \n-----------------------------\n")
            for k, v in mitre_attack_story_dict.items():
                print("[{0}] {1} {2}".format(len(list(set(v))), k, list(set(v)))) 
                
def highlight_cells_5(val):
    color= ""
    for k ,v in mitre_attack_tid_dict.items():
        if val == k and len(v) == 5:
            color = 'yellow'
            return 'background-color: {}'.format(color)

def highlight_cells_4(val):
    color= ""
    for k ,v in mitre_attack_tid_dict.items():
        if val == k and len(v) == 4:
            color = 'orange'
            return 'background-color: {}'.format(color)
        
def highlight_cells_3(val):
    color= ""
    for k ,v in mitre_attack_tid_dict.items():
        if val == k and len(v) == 3:
            color = 'lightgreen'
            return 'background-color: {}'.format(color)

def highlight_cells_2(val):
    color= ""
    for k ,v in mitre_attack_tid_dict.items():
        if val == k and len(v) == 2:
            color = 'lightblue'
            return 'background-color: {}'.format(color)

def highlight_cells_1(val):
    color= ""
    for k ,v in mitre_attack_tid_dict.items():
        if val == k and len(v) == 1:
            color = 'lightgray'
            return 'background-color: {}'.format(color)

def summary_table_of_id(dict_name, det_len): 
    temp_dict = {}  
    for k, v in dict_name.items():
        if len(v) == det_len:
           temp_dict[k] = v 
    return temp_dict

def detection_list_by_count(det_table_dict): 
    detection_count_by_id = {}
    for k, det_dict in det_table_dict.items():
        t = {}
        for s_, d in det_dict.items():
            t[s_] = "(" + str(len(d)) + ")-detection count"
        detection_count_by_id[k] = t    
    return pd.DataFrame.from_dict(detection_count_by_id, orient='index')

def detection_list_by_id(temp_dict): 
    det_table_dict = {}
    for k, v in temp_dict.items():
        temp_d = defaultdict(list)
        for fn in mitre_attack_fn_dict[k]:
            s_,d = fn.split(":")
            temp_d[s_].append(d)
        det_table_dict[k] = temp_d
    
    return det_table_dict


def highlight_cells(val):
    color= ""
    for k ,v in mitre_attack_tid_dict.items():
        if val == k and len(v) == 5:
            color = 'yellow'
            return 'background-color: {}'.format(color)
        elif val == k and len(v) == 4:
            color = 'orange'
            return 'background-color: {}'.format(color)
        elif val == k and len(v) == 3:
            color = 'lightgreen'
            return 'background-color: {}'.format(color)
        elif val == k and len(v) == 2:
            color = 'lightblue'
            return 'background-color: {}'.format(color)
        elif val == k and len(v) == 1:
            color = 'lightgray' 
            return 'background-color: {}'.format(color)

def highlight_legend(val):
    color= ""
    if val == "yellow":
        color = 'yellow'
        return 'background-color: {}'.format(color)
    elif val == 'orange':
        color = 'orange'
        return 'background-color: {}'.format(color)
    elif val == 'lightgreen':
        color = 'lightgreen'
        return 'background-color: {}'.format(color)
    elif val == 'lightblue':
        color = 'lightblue'
        return 'background-color: {}'.format(color)
    elif val == "lightgray":
        color = 'lightgray' 
        return 'background-color: {}'.format(color)

def highlight_nan(val):
    color= ""
    if val == "NaN":
        color = 'black'
        return 'background-color: {}'.format(color)
    
def show_legend():
    color_dict = {"5 malware coverage":"yellow", "4 malware coverage":"orange", "3 malware coverage":"lightgreen", "2 malware coverage":"lightblue", "1 malware coverage":"lightgray"}
    cf = pd.DataFrame.from_dict(color_dict, orient='index')
    
    return cf.style.applymap(highlight_legend)

In [4]:
def main():
    
    pd.options.display.max_colwidth = 1000
    pd.set_option('display.max_rows', 500)
    story_list = ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla"]
    process_sec_content(story_list)
    
    return

### Scan Detections
Scanning Splunk Security Content Analytic Detections related to the 5 CISA Malware family that STRT worked on.

In [5]:
if __name__ == "__main__":
    main()

 [+] story coverage needed: ['Qakbot', 'Azorult', 'Remcos', 'Trickbot', 'AgentTesla']
       [Remcos] : endpoint/powershell_windows_defender_exclusion_commands.yml
       [AgentTesla] : endpoint/powershell_windows_defender_exclusion_commands.yml
       [Remcos] : endpoint/vbscript_execution_using_wscript_app.yml
       [Azorult] : endpoint/windows_remote_service_rdpwinst_tool_execution.yml
       [Azorult] : endpoint/excessive_attempt_to_disable_services.yml
       [Qakbot] : endpoint/windows_process_injection_remote_thread.yml
       [Azorult] : endpoint/windows_remote_access_software_rms_registry.yml
       [Azorult] : endpoint/windows_remote_services_allow_remote_assistance.yml
       [Qakbot] : endpoint/system_user_discovery_with_whoami.yml
       [Trickbot] : endpoint/wermgr_process_create_executable_file.yml
       [Qakbot] : endpoint/windows_iso_lnk_file_creation.yml
       [Azorult] : endpoint/windows_iso_lnk_file_creation.yml
       [Remcos] : endpoint/windows_iso_lnk_file_cre

In [6]:
df = pd.DataFrame({ key:pd.Series(value) for key, value in mitre_attack_story_dict.items() })
df = df.sort_values(by =["Remcos", "Azorult", "Qakbot", "Trickbot", "AgentTesla"])
df
                

,Remcos,AgentTesla,Azorult,Qakbot,Trickbot
0,T1036,T1014,T1021,T1016,T1021
1,T1055,T1036,T1021.001,T1016.001,T1021.002
2,T1055.001,T1053,T1036,T1027,T1027
3,T1059,T1053.005,T1049,T1033,T1036
4,T1059.005,T1059,T1053,T1036,T1053
5,T1059.007,T1059.001,T1053.005,T1036.003,T1053.005
6,T1070,T1068,T1059,T1047,T1055
7,T1112,T1071,T1059.001,T1049,T1059
8,T1113,T1071.003,T1059.003,T1053,T1087
9,T1134,T1204,T1069,T1053.005,T1087.002


## Security Content mitre attack id Coverage for ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla"]

In [7]:
show_legend()


,0
5 malware coverage,yellow
4 malware coverage,orange
3 malware coverage,lightgreen
2 malware coverage,lightblue
1 malware coverage,lightgray


In [8]:

df.style.applymap(highlight_cells)

,Remcos,AgentTesla,Azorult,Qakbot,Trickbot
0,T1036,T1014,T1021,T1016,T1021
1,T1055,T1036,T1021.001,T1016.001,T1021.002
2,T1055.001,T1053,T1036,T1027,T1027
3,T1059,T1053.005,T1049,T1033,T1036
4,T1059.005,T1059,T1053,T1036,T1053
5,T1059.007,T1059.001,T1053.005,T1036.003,T1053.005
6,T1070,T1068,T1059,T1047,T1055
7,T1112,T1071,T1059.001,T1049,T1059
8,T1113,T1071.003,T1059.003,T1053,T1087
9,T1134,T1204,T1069,T1053.005,T1087.002


### Highlight Mitre Attack ID that detects all ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla"]

In [9]:
show_legend()

,0
5 malware coverage,yellow
4 malware coverage,orange
3 malware coverage,lightgreen
2 malware coverage,lightblue
1 malware coverage,lightgray


In [10]:
df.style.applymap(highlight_cells_5)

,Remcos,AgentTesla,Azorult,Qakbot,Trickbot
0,T1036,T1014,T1021,T1016,T1021
1,T1055,T1036,T1021.001,T1016.001,T1021.002
2,T1055.001,T1053,T1036,T1027,T1027
3,T1059,T1053.005,T1049,T1033,T1036
4,T1059.005,T1059,T1053,T1036,T1053
5,T1059.007,T1059.001,T1053.005,T1036.003,T1053.005
6,T1070,T1068,T1059,T1047,T1055
7,T1112,T1071,T1059.001,T1049,T1059
8,T1113,T1071.003,T1059.003,T1053,T1087
9,T1134,T1204,T1069,T1053.005,T1087.002


### Summary of Mitre Attack ID that detects all ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla"]

In [11]:

temp_dict = summary_table_of_id(mitre_attack_tid_dict, 5)
tdf = pd.DataFrame.from_dict(temp_dict, orient='index')
tdf

,0,1,2,3,4
T1562.001,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1562,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1059,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1566.001,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1566,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1036,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1543,AgentTesla,Azorult,Qakbot,Remcos,Trickbot


### detection list for Mitre Attack ID that detects all ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla"]

In [12]:
det_table_dict = detection_list_by_id(temp_dict)
tdf = pd.DataFrame.from_dict(det_table_dict, orient='index')
print(tdf.shape)
tdf 

(7, 5)


,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1562.001,"[add_or_set_windows_defender_exclusion.yml, excessive_usage_of_taskkill.yml, powershell_windows_defender_exclusion_commands.yml]","[attempt_to_stop_security_service.yml, disable_defender_blockatfirstseen_feature.yml, disable_defender_enhanced_notification.yml, disable_defender_spynet_reporting.yml, disable_defender_submit_samples_consent_feature.yml, disable_show_hidden_files.yml, disable_windows_behavior_monitoring.yml, excessive_usage_of_taskkill.yml, hide_user_account_from_sign_in_screen.yml, windows_defender_exclusion_registry_entry.yml, windows_disableantispyware_reg.yml, windows_impair_defense_add_xml_applocker_rules.yml, windows_impair_defense_deny_security_software_with_applocker.yml, wmic_noninteractive_app_uninstallation.yml]","[disable_defender_spynet_reporting.yml, windows_defender_exclusion_registry_entry.yml]","[add_or_set_windows_defender_exclusion.yml, powershell_windows_defender_exclusion_commands.yml, windows_defender_exclusion_registry_entry.yml]",[attempt_to_stop_security_service.yml]
T1562,"[add_or_set_windows_defender_exclusion.yml, excessive_usage_of_taskkill.yml, powershell_windows_defender_exclusion_commands.yml]","[attempt_to_stop_security_service.yml, disable_defender_blockatfirstseen_feature.yml, disable_defender_enhanced_notification.yml, disable_defender_spynet_reporting.yml, disable_defender_submit_samples_consent_feature.yml, disable_show_hidden_files.yml, disable_windows_behavior_monitoring.yml, excessive_usage_of_taskkill.yml, firewall_allowed_program_enable.yml, hide_user_account_from_sign_in_screen.yml, processes_launching_netsh.yml, windows_defender_exclusion_registry_entry.yml, windows_disableantispyware_reg.yml, windows_impair_defense_add_xml_applocker_rules.yml, windows_impair_defense_deny_security_software_with_applocker.yml, wmic_noninteractive_app_uninstallation.yml]","[disable_defender_spynet_reporting.yml, windows_defender_exclusion_registry_entry.yml]","[add_or_set_windows_defender_exclusion.yml, powershell_windows_defender_exclusion_commands.yml, windows_defender_exclusion_registry_entry.yml]",[attempt_to_stop_security_service.yml]
T1059,"[powershell___connect_to_internet_with_hidden_window.yml, powershell_loading_dotnet_into_memory_via_reflection.yml]","[chcp_command_execution.yml, cmd_carry_out_string_command_parameter.yml, detect_use_of_cmd_exe_to_launch_script_interpreters.yml]","[cmd_carry_out_string_command_parameter.yml, cmdline_tool_not_executed_in_cmd_shell.yml, wermgr_process_spawned_cmd_or_powershell_process.yml]","[jscript_execution_using_cscript_app.yml, process_writing_dynamicwrapperx.yml, suspicious_process_dns_query_known_abuse_web_services.yml, vbscript_execution_using_wscript_app.yml]",[wermgr_process_spawned_cmd_or_powershell_process.yml]
T1566.001,"[office_application_drop_executable.yml, office_application_spawn_rundll32_process.yml, office_document_executing_macro_code.yml, office_product_spawn_cmd_process.yml, office_product_spawning_certutil.yml, windows_iso_lnk_file_creation.yml, windows_phishing_recent_iso_exec_registry.yml]","[office_document_executing_macro_code.yml, office_product_spawn_cmd_process.yml, office_product_spawning_mshta.yml, windows_iso_lnk_file_creation.yml, windows_phishing_recent_iso_exec_registry.yml]","[office_application_spawn_regsvr32_process.yml, office_document_executing_macro_code.yml, office_product_spawn_cmd_process.yml, windows_iso_lnk_file_creation.yml, windows_phishing_recent_iso_exec_registry.yml]","[office_document_executing_macro_code.yml, office_product_spawn_cmd_process.yml, office_product_spawning_windows_script_host.yml, windows_iso_lnk_file_creation.yml, windows_phishing_recent_iso_exec_registry.yml]","[office_application_spawn_rundll32_process.yml, office_document_executing_macro_code.yml, office_product_spawn_cmd_process.yml, office_product_spawning_certutil.yml]"
T1566,"[office_application_drop_executable.yml, office_application_spawn_rundll32_process.yml, offi

### detection count per mitre attack id in each malware family

In [13]:
detection_list_by_count(det_table_dict)

,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1562.001,(3)-detection count,(14)-detection count,(2)-detection count,(3)-detection count,(1)-detection count
T1562,(3)-detection count,(16)-detection count,(2)-detection count,(3)-detection count,(1)-detection count
T1059,(2)-detection count,(3)-detection count,(3)-detection count,(4)-detection count,(1)-detection count
T1566.001,(7)-detection count,(5)-detection count,(5)-detection count,(5)-detection count,(4)-detection count
T1566,(7)-detection count,(5)-detection count,(6)-detection count,(5)-detection count,(4)-detection count
T1036,(1)-detection count,(1)-detection count,(3)-detection count,(1)-detection count,(1)-detection count
T1543,(2)-detection count,(2)-detection count,(2)-detection count,(2)-detection count,(1)-detection count


### Highlight Mitre Attack ID that detects 4/5 malware family

In [14]:
show_legend()

,0
5 malware coverage,yellow
4 malware coverage,orange
3 malware coverage,lightgreen
2 malware coverage,lightblue
1 malware coverage,lightgray


In [15]:
df.style.applymap(highlight_cells_4)

,Remcos,AgentTesla,Azorult,Qakbot,Trickbot
0,T1036,T1014,T1021,T1016,T1021
1,T1055,T1036,T1021.001,T1016.001,T1021.002
2,T1055.001,T1053,T1036,T1027,T1027
3,T1059,T1053.005,T1049,T1033,T1036
4,T1059.005,T1059,T1053,T1036,T1053
5,T1059.007,T1059.001,T1053.005,T1036.003,T1053.005
6,T1070,T1068,T1059,T1047,T1055
7,T1112,T1071,T1059.001,T1049,T1059
8,T1113,T1071.003,T1059.003,T1053,T1087
9,T1134,T1204,T1069,T1053.005,T1087.002


### Summary of Mitre Attack ID that detects 4/5 malware family

In [16]:
temp_dict = summary_table_of_id(mitre_attack_tid_dict, 4)
tdf = pd.DataFrame.from_dict(temp_dict, orient='index')
tdf = tdf.sort_values(by=[0,1,2,3])
tdf

,0,1,2,3
T1204.001,AgentTesla,Azorult,Qakbot,Remcos
T1204,AgentTesla,Azorult,Qakbot,Remcos
T1053,AgentTesla,Azorult,Qakbot,Trickbot
T1053.005,AgentTesla,Azorult,Qakbot,Trickbot
T1218,AgentTesla,Qakbot,Remcos,Trickbot


### detection list for Mitre Attack ID that detects 4/5 malware families

In [17]:
det_table_dict = detection_list_by_id(temp_dict)
tdf = pd.DataFrame.from_dict(det_table_dict, orient='index')
print(tdf.shape)
tdf  

(5, 5)


,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1204.001,[windows_iso_lnk_file_creation.yml],[windows_iso_lnk_file_creation.yml],[windows_iso_lnk_file_creation.yml],[windows_iso_lnk_file_creation.yml],NaN
T1204,[windows_iso_lnk_file_creation.yml],[windows_iso_lnk_file_creation.yml],[windows_iso_lnk_file_creation.yml],[windows_iso_lnk_file_creation.yml],NaN
T1218,[detect_html_help_spawn_child_process.yml],NaN,"[regsvr32_with_known_silent_switch_cmdline.yml, suspicious_regsvr32_register_suspicious_path.yml, windows_regsvr32_renamed_binary.yml]","[regsvr32_silent_and_install_param_dll_loading.yml, regsvr32_with_known_silent_switch_cmdline.yml]","[mshta_spawning_rundll32_or_regsvr32_process.yml, suspicious_rundll32_startw.yml]"
T1053,[scheduled_task_deleted_or_created_via_cmd.yml],"[scheduled_task_deleted_or_created_via_cmd.yml, suspicious_scheduled_task_from_public_directory.yml]","[scheduled_task_deleted_or_created_via_cmd.yml, schtasks_run_task_on_demand.yml, windows_schtasks_create_run_as_system.yml]",NaN,"[schedule_task_with_rundll32_command_trigger.yml, scheduled_task_deleted_or_created_via_cmd.yml]"
T1053.005,[scheduled_task_deleted_or_created_via_cmd.yml],"[scheduled_task_deleted_or_created_via_cmd.yml, suspicious_scheduled_task_from_public_directory.yml]","[scheduled_task_deleted_or_created_via_cmd.yml, windows_schtasks_create_run_as_system.yml, winevent_windows_task_scheduler_event_action_started.yml]",NaN,[scheduled_task_deleted_or_created_via_cmd.yml]


### detection count per mitre attack id in each malware family

In [18]:
detection_list_by_count(det_table_dict)

,AgentTesla,Azorult,Qakbot,Remcos,Trickbot
T1204.001,(1)-detection count,(1)-detection count,(1)-detection count,(1)-detection count,NaN
T1204,(1)-detection count,(1)-detection count,(1)-detection count,(1)-detection count,NaN
T1218,(1)-detection count,NaN,(3)-detection count,(2)-detection count,(2)-detection count
T1053,(1)-detection count,(2)-detection count,(3)-detection count,NaN,(2)-detection count
T1053.005,(1)-detection count,(2)-detection count,(3)-detection count,NaN,(1)-detection count


### Highlight Mitre Attack ID that detects 3/5 malware family

In [19]:
show_legend()

,0
5 malware coverage,yellow
4 malware coverage,orange
3 malware coverage,lightgreen
2 malware coverage,lightblue
1 malware coverage,lightgray


In [20]:
df.style.applymap(highlight_cells_3)

,Remcos,AgentTesla,Azorult,Qakbot,Trickbot
0,T1036,T1014,T1021,T1016,T1021
1,T1055,T1036,T1021.001,T1016.001,T1021.002
2,T1055.001,T1053,T1036,T1027,T1027
3,T1059,T1053.005,T1049,T1033,T1036
4,T1059.005,T1059,T1053,T1036,T1053
5,T1059.007,T1059.001,T1053.005,T1036.003,T1053.005
6,T1070,T1068,T1059,T1047,T1055
7,T1112,T1071,T1059.001,T1049,T1059
8,T1113,T1071.003,T1059.003,T1053,T1087
9,T1134,T1204,T1069,T1053.005,T1087.002


### Summary of Mitre Attack ID that detects 3/5 malware family

In [21]:

temp_dict = summary_table_of_id(mitre_attack_tid_dict, 3)
tdf = pd.DataFrame.from_dict(temp_dict, orient='index')
tdf = tdf.sort_values(by=[0,1,2])
tdf

,0,1,2
T1071,AgentTesla,Azorult,Qakbot
T1059.001,AgentTesla,Azorult,Qakbot
T1543.003,AgentTesla,Azorult,Qakbot
T1548,AgentTesla,Azorult,Remcos
T1555.003,AgentTesla,Azorult,Remcos
T1555,AgentTesla,Azorult,Remcos
T1548.002,AgentTesla,Azorult,Remcos
T1112,Azorult,Qakbot,Remcos
T1547.001,Azorult,Qakbot,Remcos
T1547,Azorult,Qakbot,Remcos


### detection list for Mitre Attack ID that detects 3/5 malware families

In [22]:
det_table_dict = detection_list_by_id(temp_dict)
tdf = pd.DataFrame.from_dict(det_table_dict, orient='index')
print(tdf.shape)
tdf  

(11, 5)


,Qakbot,Remcos,Trickbot,AgentTesla,Azorult
T1055,"[create_remote_thread_in_shell_application.yml, windows_command_shell_fetch_env_variables.yml, windows_process_injection_of_wermgr_to_known_browser.yml, windows_process_injection_remote_thread.yml, windows_process_injection_wermgr_child_process.yml]","[loading_of_dynwrapx_module.yml, winhlp32_spawning_a_process.yml, wscript_or_cscript_suspicious_child_process.yml]","[cobalt_strike_named_pipes.yml, powershell_remote_thread_to_known_windows_process.yml, trickbot_named_pipe.yml]",NaN,NaN
T1071,"[windows_app_layer_protocol_qakbot_namedpipe.yml, windows_app_layer_protocol_wermgr_connect_to_namedpipe.yml]",NaN,NaN,"[windows_file_transfer_protocol_in_non_common_process_path.yml, windows_mail_protocol_in_non_common_process_path.yml, windows_multi_hop_proxy_tor_website_query.yml]",[windows_application_layer_protocol_rms_radmin_tool_namedpipe.yml]
T1112,[windows_modify_registry_qakbot_binary_data_registry.yml],"[malicious_inprocserver32_modification.yml, remcos_client_registry_install_entry.yml]",NaN,NaN,"[windows_modify_registry_disable_toast_notifications.yml, windows_modify_registry_disable_win_defender_raw_write_notif.yml, windows_modify_registry_disable_windows_security_center_notif.yml, windows_modify_registry_disabling_wer_settings.yml, windows_modify_registry_disallow_windows_app.yml, windows_modify_registry_regedit_silent_reg_import.yml, windows_modify_registry_suppress_win_defender_notif.yml]"
T1059.001,[recon_using_wmi_class.yml],NaN,NaN,"[powershell___connect_to_internet_with_hidden_window.yml, powershell_loading_dotnet_into_memory_via_reflection.yml]",[windows_powershell_import_applocker_policy.yml]
T1543.003,[services_exe_lolbas_execution_process_spawn.yml],NaN,NaN,[suspicious_driver_loaded_path.yml],[sc_exe_manipulating_windows_services.yml]
T1547.001,[registry_keys_used_for_persistence.yml],[registry_keys_used_for_persistence.yml],NaN,NaN,[registry_keys_used_for_persistence.yml]
T1547,[registry_keys_used_for_persistence.yml],[registry_keys_used_for_persistence.yml],NaN,NaN,[registry_keys_used_for_persistence.yml]
T1548,NaN,[disabling_remote_user_account_control.yml],NaN,[disabling_remote_user_account_control.yml],"[allow_operation_with_consent_admin.yml, disabling_remote_user_account_control.yml]"
T1555.003,NaN,"[non_chrome_process_accessing_chrome_default_dir.yml, non_firefox_process_access_firefox_profile_dir.yml, possible_browser_pass_view_parameter.yml]",NaN,"[non_chrome_process_accessing_chrome_default_dir.yml, non_firefox_process_access_firefox_profile_dir.yml]",[non_firefox_process_access_firefox_profile_dir.yml]
T1555,NaN,"[non_chrome_process_accessing_chrome_default_dir.yml, non_firefox_process_access_firefox_profile_dir.yml, possible_browser_pass_view_parameter.yml]",NaN,"[non_chrome_process_accessing_chrome_default_dir.yml, non_firefox_process_access_firefox_profile_dir.yml]",[non_firefox_process_access_firefox_profile_dir.yml]


### detection count per mitre attack id in each malware family

In [23]:
detection_list_by_count(det_table_dict)

,Qakbot,Remcos,Trickbot,AgentTesla,Azorult
T1055,(5)-detection count,(3)-detection count,(3)-detection count,NaN,NaN
T1071,(2)-detection count,NaN,NaN,(3)-detection count,(1)-detection count
T1112,(1)-detection count,(2)-detection count,NaN,NaN,(7)-detection count
T1059.001,(1)-detection count,NaN,NaN,(2)-detection count,(1)-detection count
T1543.003,(1)-detection count,NaN,NaN,(1)-detection count,(1)-detection count
T1547.001,(1)-detection count,(1)-detection count,NaN,NaN,(1)-detection count
T1547,(1)-detection count,(1)-detection count,NaN,NaN,(1)-detection count
T1548,NaN,(1)-detection count,NaN,(1)-detection count,(2)-detection count
T1555.003,NaN,(3)-detection count,NaN,(2)-detection count,(1)-detection count
T1555,NaN,(3)-detection count,NaN,(2)-detection count,(1)-detection count


### Highlight Mitre Attack ID that detects 2/5 malware family

In [24]:
show_legend()

,0
5 malware coverage,yellow
4 malware coverage,orange
3 malware coverage,lightgreen
2 malware coverage,lightblue
1 malware coverage,lightgray


In [25]:
df.style.applymap(highlight_cells_2)

,Remcos,AgentTesla,Azorult,Qakbot,Trickbot
0,T1036,T1014,T1021,T1016,T1021
1,T1055,T1036,T1021.001,T1016.001,T1021.002
2,T1055.001,T1053,T1036,T1027,T1027
3,T1059,T1053.005,T1049,T1033,T1036
4,T1059.005,T1059,T1053,T1036,T1053
5,T1059.007,T1059.001,T1053.005,T1036.003,T1053.005
6,T1070,T1068,T1059,T1047,T1055
7,T1112,T1071,T1059.001,T1049,T1059
8,T1113,T1071.003,T1059.003,T1053,T1087
9,T1134,T1204,T1069,T1053.005,T1087.002


### Summary of Mitre Attack ID that detects 2/5 malware family

In [26]:
temp_dict = summary_table_of_id(mitre_attack_tid_dict, 2)
tdf = pd.DataFrame.from_dict(temp_dict, orient='index')
tdf = tdf.sort_values(by=[0,1])
tdf

,0,1
T1049,Azorult,Qakbot
T1059.003,Azorult,Qakbot
T1569,Azorult,Qakbot
T1569.002,Azorult,Qakbot
T1021,Azorult,Trickbot
T1590,Azorult,Trickbot
T1590.005,Azorult,Trickbot
T1059.007,Qakbot,Remcos
T1218.010,Qakbot,Remcos
T1055.001,Qakbot,Remcos


### detection list for Mitre Attack ID that detects 2/5 malware families

In [27]:
det_table_dict = detection_list_by_id(temp_dict)
tdf = pd.DataFrame.from_dict(det_table_dict, orient='index')
print(tdf.shape)
tdf 

(12, 4)


,Azorult,Trickbot,Qakbot,Remcos
T1021,"[allow_inbound_traffic_by_firewall_rule_registry.yml, windows_remote_service_rdpwinst_tool_execution.yml, windows_remote_services_allow_rdp_in_firewall.yml, windows_remote_services_allow_remote_assistance.yml, windows_remote_services_rdp_enable.yml]",[executable_file_written_in_administrative_smb_share.yml],NaN,NaN
T1590,[windows_gather_victim_network_info_through_ip_check_web_services.yml],[wermgr_process_connecting_to_ip_check_web_services.yml],NaN,NaN
T1590.005,[windows_gather_victim_network_info_through_ip_check_web_services.yml],[wermgr_process_connecting_to_ip_check_web_services.yml],NaN,NaN
T1049,[network_connection_discovery_net.yml],NaN,"[network_connection_discovery_arp.yml, network_connection_discovery_netstat.yml]",NaN
T1059.003,"[cmd_carry_out_string_command_parameter.yml, detect_use_of_cmd_exe_to_launch_script_interpreters.yml]",NaN,[cmd_carry_out_string_command_parameter.yml],NaN
T1569,[excessive_usage_of_sc_service_utility.yml],NaN,[windows_service_created_with_suspicious_service_path.yml],NaN
T1569.002,[excessive_usage_of_sc_service_utility.yml],NaN,[windows_service_created_with_suspicious_service_path.yml],NaN
T1027,NaN,[wermgr_process_create_executable_file.yml],[malicious_powershell_process___encoded_command.yml],NaN
T1059.007,NaN,NaN,[cmdline_tool_not_executed_in_cmd_shell.yml],[jscript_execution_using_cscript_app.yml]
T1218.010,NaN,NaN,"[regsvr32_with_known_silent_switch_cmdline.yml, suspicious_regsvr32_register_suspicious_path.yml, windows_regsvr32_renamed_binary.yml]","[malicious_inprocserver32_modification.yml, regsvr32_silent_and_install_param_dll_loading.yml, regsvr32_with_known_silent_switch_cmdline.yml]"


### detection count per mitre attack id in each malware family

In [28]:
detection_list_by_count(det_table_dict)

,Azorult,Trickbot,Qakbot,Remcos
T1021,(5)-detection count,(1)-detection count,NaN,NaN
T1590,(1)-detection count,(1)-detection count,NaN,NaN
T1590.005,(1)-detection count,(1)-detection count,NaN,NaN
T1049,(1)-detection count,NaN,(2)-detection count,NaN
T1059.003,(2)-detection count,NaN,(1)-detection count,NaN
T1569,(1)-detection count,NaN,(1)-detection count,NaN
T1569.002,(1)-detection count,NaN,(1)-detection count,NaN
T1027,NaN,(1)-detection count,(1)-detection count,NaN
T1059.007,NaN,NaN,(1)-detection count,(1)-detection count
T1218.010,NaN,NaN,(3)-detection count,(3)-detection count


### Highlight Mitre Attack ID that detects 1/5 malware family

In [29]:
show_legend()

,0
5 malware coverage,yellow
4 malware coverage,orange
3 malware coverage,lightgreen
2 malware coverage,lightblue
1 malware coverage,lightgray


In [30]:
df.style.applymap(highlight_cells_1)

,Remcos,AgentTesla,Azorult,Qakbot,Trickbot
0,T1036,T1014,T1021,T1016,T1021
1,T1055,T1036,T1021.001,T1016.001,T1021.002
2,T1055.001,T1053,T1036,T1027,T1027
3,T1059,T1053.005,T1049,T1033,T1036
4,T1059.005,T1059,T1053,T1036,T1053
5,T1059.007,T1059.001,T1053.005,T1036.003,T1053.005
6,T1070,T1068,T1059,T1047,T1055
7,T1112,T1071,T1059.001,T1049,T1059
8,T1113,T1071.003,T1059.003,T1053,T1087
9,T1134,T1204,T1069,T1053.005,T1087.002


### Summary of Mitre Attack ID that detects 1/5 malware family

In [31]:
temp_dict = summary_table_of_id(mitre_attack_tid_dict, 1)
tdf = pd.DataFrame.from_dict(temp_dict, orient='index')
tdf = tdf.sort_values(by=[0])
tdf

,0
T1218.001,AgentTesla
T1068,AgentTesla
T1071.003,AgentTesla
T1014,AgentTesla
T1136,Azorult
T1222.001,Azorult
T1564,Azorult
T1564.001,Azorult
T1531,Azorult
T1069,Azorult


### detection list for Mitre Attack ID that detects 1/5 malware families

In [32]:
det_table_dict = detection_list_by_id(temp_dict)
tdf = pd.DataFrame.from_dict(det_table_dict, orient='index')
print(tdf.shape)
tdf 

(39, 5)


,Remcos,Azorult,Qakbot,AgentTesla,Trickbot
T1059.005,"[suspicious_process_dns_query_known_abuse_web_services.yml, vbscript_execution_using_wscript_app.yml]",NaN,NaN,NaN,NaN
T1113,"[remcos_rat_file_creation_in_remcos_folder.yml, suspicious_image_creation_in_appdata_folder.yml, suspicious_wav_file_in_appdata_folder.yml]",NaN,NaN,NaN,NaN
T1070,[process_deleting_its_process_file_path.yml],NaN,NaN,NaN,NaN
T1559.001,[process_writing_dynamicwrapperx.yml],NaN,NaN,NaN,NaN
T1134.004,[wscript_or_cscript_suspicious_child_process.yml],NaN,NaN,NaN,NaN
T1134,[wscript_or_cscript_suspicious_child_process.yml],NaN,NaN,NaN,NaN
T1021.001,NaN,"[allow_inbound_traffic_by_firewall_rule_registry.yml, windows_remote_service_rdpwinst_tool_execution.yml, windows_remote_services_allow_rdp_in_firewall.yml, windows_remote_services_allow_remote_assistance.yml, windows_remote_services_rdp_enable.yml]",NaN,NaN,NaN
T1489,NaN,"[excessive_attempt_to_disable_services.yml, windows_service_stop_by_deletion.yml, windows_valid_account_with_never_expires_password.yml]",NaN,NaN,NaN
T1219,NaN,[windows_remote_access_software_rms_registry.yml],NaN,NaN,NaN
T1562.004,NaN,"[firewall_allowed_program_enable.yml, processes_launching_netsh.yml]",NaN,NaN,NaN


### detection count per mitre attack id in each malware family

In [33]:
detection_list_by_count(det_table_dict)

,Remcos,Azorult,Qakbot,AgentTesla,Trickbot
T1059.005,(2)-detection count,NaN,NaN,NaN,NaN
T1113,(3)-detection count,NaN,NaN,NaN,NaN
T1070,(1)-detection count,NaN,NaN,NaN,NaN
T1559.001,(1)-detection count,NaN,NaN,NaN,NaN
T1134.004,(1)-detection count,NaN,NaN,NaN,NaN
T1134,(1)-detection count,NaN,NaN,NaN,NaN
T1021.001,NaN,(5)-detection count,NaN,NaN,NaN
T1489,NaN,(3)-detection count,NaN,NaN,NaN
T1219,NaN,(1)-detection count,NaN,NaN,NaN
T1562.004,NaN,(2)-detection count,NaN,NaN,NaN


In [34]:
detection_tid_list =[]
for k,v in mitre_attack_tid_dict.items():
    detection_tid_list.append(k)

print(sorted(detection_tid_list))

['T1014', 'T1016', 'T1016.001', 'T1021', 'T1021.001', 'T1021.002', 'T1027', 'T1033', 'T1036', 'T1036.003', 'T1047', 'T1049', 'T1053', 'T1053.005', 'T1055', 'T1055.001', 'T1055.002', 'T1059', 'T1059.001', 'T1059.003', 'T1059.005', 'T1059.007', 'T1068', 'T1069', 'T1069.001', 'T1070', 'T1071', 'T1071.003', 'T1087', 'T1087.002', 'T1112', 'T1113', 'T1134', 'T1134.004', 'T1136', 'T1136.001', 'T1204', 'T1204.001', 'T1218', 'T1218.001', 'T1218.005', 'T1218.010', 'T1218.011', 'T1219', 'T1222', 'T1222.001', 'T1482', 'T1489', 'T1531', 'T1543', 'T1543.003', 'T1547', 'T1547.001', 'T1548', 'T1548.002', 'T1555', 'T1555.003', 'T1559.001', 'T1562', 'T1562.001', 'T1562.004', 'T1564', 'T1564.001', 'T1566', 'T1566.001', 'T1566.002', 'T1569', 'T1569.002', 'T1574', 'T1574.001', 'T1574.002', 'T1590', 'T1590.005', 'T1592']


### truth table splunk detection name vs analytic story (malware family)

In [35]:
def yes_no_table(dict_data, story_name_list):
    det_by_analytics = defaultdict(list)

    for k, v in dict_data.items():
        found_list = []
        det_ctr = 0
        for st in story_name_list:
            if st in v:
                found_list.append("yes")
                det_by_analytics[k].append("yes")
                det_ctr += 1
            else:
                found_list.append("no")
                det_by_analytics[k].append("no")
        det_by_analytics[k].append(det_ctr)

    return det_by_analytics

def highlight_rows(x):
    if x.total_coverage == 1:
        return['background-color: lightgray'] * 6
    elif x.total_coverage == 2:
        return['background-color: lightblue'] * 6
    elif x.total_coverage == 3:
        return['background-color: lightgreen'] * 6    
    elif x.total_coverage == 4:
        return['background-color: orange'] * 6  
    elif x.total_coverage == 5:
        return['background-color: yellow'] * 6  
story_name_list = ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla"]
det_by_analytics = yes_no_table(mitre_attack_id_info_dict, story_name_list)
tdf = pd.DataFrame.from_dict(det_by_analytics, orient="index")
tdf.columns = ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla", "total_coverage"]
print(tdf.shape)
tdf = tdf.sort_values(by=['total_coverage'], ascending = False)
tdf.style.apply(highlight_rows, axis=1)


(140, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
office_document_executing_macro_code.yml,yes,yes,yes,yes,yes,5
suspicious_process_file_path.yml,yes,yes,yes,yes,yes,5
executables_or_script_creation_in_suspicious_path.yml,yes,yes,yes,yes,yes,5
office_product_spawn_cmd_process.yml,yes,yes,yes,yes,yes,5
windows_phishing_recent_iso_exec_registry.yml,yes,yes,yes,no,yes,4
windows_iso_lnk_file_creation.yml,yes,yes,yes,no,yes,4
scheduled_task_deleted_or_created_via_cmd.yml,yes,yes,no,yes,yes,4
registry_keys_used_for_persistence.yml,yes,yes,yes,no,no,3
non_firefox_process_access_firefox_profile_dir.yml,no,yes,yes,no,yes,3
windows_defender_exclusion_registry_entry.yml,yes,yes,yes,no,no,3


### breakdown of detection coverage for 5 malware family or story

In [36]:
rslt_df = tdf[tdf['total_coverage'] == 5]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)


(4, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
office_document_executing_macro_code.yml,yes,yes,yes,yes,yes,5
suspicious_process_file_path.yml,yes,yes,yes,yes,yes,5
executables_or_script_creation_in_suspicious_path.yml,yes,yes,yes,yes,yes,5
office_product_spawn_cmd_process.yml,yes,yes,yes,yes,yes,5


In [37]:
rslt_df = tdf[tdf['total_coverage'] == 4]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(3, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
windows_phishing_recent_iso_exec_registry.yml,yes,yes,yes,no,yes,4
windows_iso_lnk_file_creation.yml,yes,yes,yes,no,yes,4
scheduled_task_deleted_or_created_via_cmd.yml,yes,yes,no,yes,yes,4


In [38]:
rslt_df = tdf[tdf['total_coverage'] == 3]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(4, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
registry_keys_used_for_persistence.yml,yes,yes,yes,no,no,3
non_firefox_process_access_firefox_profile_dir.yml,no,yes,yes,no,yes,3
windows_defender_exclusion_registry_entry.yml,yes,yes,yes,no,no,3
disabling_remote_user_account_control.yml,no,yes,yes,no,yes,3


In [39]:
rslt_df = tdf[tdf['total_coverage'] == 2]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(11, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
add_or_set_windows_defender_exclusion.yml,no,no,yes,no,yes,2
disable_defender_spynet_reporting.yml,yes,yes,no,no,no,2
office_product_spawning_certutil.yml,no,no,no,yes,yes,2
excessive_usage_of_taskkill.yml,no,yes,no,no,yes,2
regsvr32_with_known_silent_switch_cmdline.yml,yes,no,yes,no,no,2
cmd_carry_out_string_command_parameter.yml,yes,yes,no,no,no,2
office_application_spawn_rundll32_process.yml,no,no,no,yes,yes,2
powershell_windows_defender_exclusion_commands.yml,no,no,yes,no,yes,2
attempt_to_stop_security_service.yml,no,yes,no,yes,no,2
wermgr_process_spawned_cmd_or_powershell_process.yml,yes,no,no,yes,no,2


In [40]:
rslt_df = tdf[tdf['total_coverage'] == 1]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(118, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
chcp_command_execution.yml,no,yes,no,no,no,1
jscript_execution_using_cscript_app.yml,no,no,yes,no,no,1
wmic_noninteractive_app_uninstallation.yml,no,yes,no,no,no,1
executable_file_written_in_administrative_smb_share.yml,no,no,no,yes,no,1
windows_powershell_import_applocker_policy.yml,no,yes,no,no,no,1
windows_service_stop_by_deletion.yml,no,yes,no,no,no,1
disable_defender_submit_samples_consent_feature.yml,no,yes,no,no,no,1
schedule_task_with_rundll32_command_trigger.yml,no,no,no,yes,no,1
suspicious_copy_on_system32.yml,yes,no,no,no,no,1
detect_html_help_spawn_child_process.yml,no,no,no,no,yes,1


### truth table mitre attack id vs analytic story (malware family)

In [41]:

new_table = yes_no_table(mitre_attack_tid_dict, story_name_list)
tdf = pd.DataFrame.from_dict(new_table, orient="index")
tdf.columns = ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla", "total_coverage"]
print(tdf.shape)
tdf = tdf.sort_values(by=['total_coverage'], ascending = False)
tdf.style.apply(highlight_rows, axis=1) 


(74, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1562.001,yes,yes,yes,yes,yes,5
T1036,yes,yes,yes,yes,yes,5
T1566,yes,yes,yes,yes,yes,5
T1566.001,yes,yes,yes,yes,yes,5
T1562,yes,yes,yes,yes,yes,5
T1543,yes,yes,yes,yes,yes,5
T1059,yes,yes,yes,yes,yes,5
T1053.005,yes,yes,no,yes,yes,4
T1204.001,yes,yes,yes,no,yes,4
T1204,yes,yes,yes,no,yes,4


In [42]:
rslt_df = tdf[tdf['total_coverage'] == 5]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(7, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1562.001,yes,yes,yes,yes,yes,5
T1036,yes,yes,yes,yes,yes,5
T1566,yes,yes,yes,yes,yes,5
T1566.001,yes,yes,yes,yes,yes,5
T1562,yes,yes,yes,yes,yes,5
T1543,yes,yes,yes,yes,yes,5
T1059,yes,yes,yes,yes,yes,5


In [43]:
rslt_df = tdf[tdf['total_coverage'] == 4]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(5, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1053.005,yes,yes,no,yes,yes,4
T1204.001,yes,yes,yes,no,yes,4
T1204,yes,yes,yes,no,yes,4
T1053,yes,yes,no,yes,yes,4
T1218,yes,no,yes,yes,yes,4


In [44]:
rslt_df = tdf[tdf['total_coverage'] == 3]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(11, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1543.003,yes,yes,no,no,yes,3
T1548.002,no,yes,yes,no,yes,3
T1059.001,yes,yes,no,no,yes,3
T1548,no,yes,yes,no,yes,3
T1555,no,yes,yes,no,yes,3
T1547.001,yes,yes,yes,no,no,3
T1555.003,no,yes,yes,no,yes,3
T1112,yes,yes,yes,no,no,3
T1071,yes,yes,no,no,yes,3
T1547,yes,yes,yes,no,no,3


In [45]:
rslt_df = tdf[tdf['total_coverage'] == 2]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(12, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1049,yes,yes,no,no,no,2
T1592,yes,no,yes,no,no,2
T1021,no,yes,no,yes,no,2
T1027,yes,no,no,yes,no,2
T1055.001,yes,no,yes,no,no,2
T1590,no,yes,no,yes,no,2
T1059.003,yes,yes,no,no,no,2
T1218.010,yes,no,yes,no,no,2
T1590.005,no,yes,no,yes,no,2
T1059.007,yes,no,yes,no,no,2


In [46]:
rslt_df = tdf[tdf['total_coverage'] == 1]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(39, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1087.002,no,no,no,yes,no,1
T1087,no,no,no,yes,no,1
T1218.011,no,no,no,yes,no,1
T1218.001,no,no,no,no,yes,1
T1021.002,no,no,no,yes,no,1
T1564.001,no,yes,no,no,no,1
T1482,yes,no,no,no,no,1
T1564,no,yes,no,no,no,1
T1559.001,no,no,yes,no,no,1
T1566.002,yes,no,no,no,no,1


### truth table mitre attack id 0technique vs analytic story(malware family)

In [47]:


new_table = yes_no_table(mitre_attack_tid_name_dict, story_name_list)
tdf = pd.DataFrame.from_dict(new_table, orient="index")
tdf.columns = ["Qakbot", "Azorult", "Remcos", "Trickbot", "AgentTesla", "total_coverage"]
print(tdf.shape)
tdf = tdf.sort_values(by=['total_coverage'], ascending = False)
tdf.style.apply(highlight_rows, axis=1) 

(74, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1562.001-Disable_or_Modify_Tools,yes,yes,yes,yes,yes,5
T1036-Masquerading (7),yes,yes,yes,yes,yes,5
T1566-Phishing (3),yes,yes,yes,yes,yes,5
T1566.001-Spearphishing_Attachment,yes,yes,yes,yes,yes,5
T1562-Impair_Defenses (9),yes,yes,yes,yes,yes,5
T1543-Create_or_Modify_System_Process (4),yes,yes,yes,yes,yes,5
T1059-Command_and_Scripting_Interpreter (8),yes,yes,yes,yes,yes,5
T1053.005-Scheduled_Task,yes,yes,no,yes,yes,4
T1204.001-Malicious_Link,yes,yes,yes,no,yes,4
T1204-User_Execution (3),yes,yes,yes,no,yes,4


In [48]:
rslt_df = tdf[tdf['total_coverage'] == 5]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(7, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1562.001-Disable_or_Modify_Tools,yes,yes,yes,yes,yes,5
T1036-Masquerading (7),yes,yes,yes,yes,yes,5
T1566-Phishing (3),yes,yes,yes,yes,yes,5
T1566.001-Spearphishing_Attachment,yes,yes,yes,yes,yes,5
T1562-Impair_Defenses (9),yes,yes,yes,yes,yes,5
T1543-Create_or_Modify_System_Process (4),yes,yes,yes,yes,yes,5
T1059-Command_and_Scripting_Interpreter (8),yes,yes,yes,yes,yes,5


In [49]:
rslt_df = tdf[tdf['total_coverage'] == 4]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(5, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1053.005-Scheduled_Task,yes,yes,no,yes,yes,4
T1204.001-Malicious_Link,yes,yes,yes,no,yes,4
T1204-User_Execution (3),yes,yes,yes,no,yes,4
T1053-Scheduled_Task/Job (5),yes,yes,no,yes,yes,4
T1218-System_Binary_Proxy_Execution (13),yes,no,yes,yes,yes,4


In [50]:
rslt_df = tdf[tdf['total_coverage'] == 3]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(11, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1543.003-Windows_Service,yes,yes,no,no,yes,3
T1548.002-Bypass_User_Account_Control,no,yes,yes,no,yes,3
T1059.001-PowerShell,yes,yes,no,no,yes,3
T1548-Abuse_Elevation_Control_Mechanism (4),no,yes,yes,no,yes,3
T1555-Credentials_from_Password_Stores (5),no,yes,yes,no,yes,3
T1547.001-Registry_Run_Keys_/_Startup_Folder,yes,yes,yes,no,no,3
T1555.003-Credentials_from_Web_Browsers,no,yes,yes,no,yes,3
T1112-Modify_Registry,yes,yes,yes,no,no,3
T1071-Application_Layer_Protocol (4),yes,yes,no,no,yes,3
T1547-Boot_or_Logon_Autostart_Execution (14),yes,yes,yes,no,no,3


In [51]:
rslt_df = tdf[tdf['total_coverage'] == 2]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(12, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1049-System_Network_Connections_Discovery,yes,yes,no,no,no,2
T1592-Gather_Victim_Host_Information (4),yes,no,yes,no,no,2
T1021-Remote_Services (6),no,yes,no,yes,no,2
T1027-Obfuscated_Files_or_Information (9),yes,no,no,yes,no,2
T1055.001-Dynamic-link_Library_Injection,yes,no,yes,no,no,2
T1590-Gather_Victim_Network_Information (6),no,yes,no,yes,no,2
T1059.003-Windows_Command_Shell,yes,yes,no,no,no,2
T1218.010-Regsvr32,yes,no,yes,no,no,2
T1590.005-IP_Addresses,no,yes,no,yes,no,2
T1059.007-JavaScript,yes,no,yes,no,no,2


In [52]:
rslt_df = tdf[tdf['total_coverage'] == 1]
print(rslt_df.shape)
rslt_df.style.apply(highlight_rows, axis=1)

(39, 6)


,Qakbot,Azorult,Remcos,Trickbot,AgentTesla,total_coverage
T1087.002-Domain_Account,no,no,no,yes,no,1
T1087-Account_Discovery (4),no,no,no,yes,no,1
T1218.011-Rundll32,no,no,no,yes,no,1
T1218.001-Compiled_HTML_File,no,no,no,no,yes,1
T1021.002-SMB/Windows_Admin_Shares,no,no,no,yes,no,1
T1564.001-Hidden_Files_and_Directories,no,yes,no,no,no,1
T1482-Domain_Trust_Discovery,yes,no,no,no,no,1
T1564-Hide_Artifacts (10),no,yes,no,no,no,1
T1559.001-Component_Object_Model,no,no,yes,no,no,1
T1566.002-Spearphishing_Link,yes,no,no,no,no,1
